In [1]:
import json

In [3]:
f = open('data.json')

data = json.load(f)

In [21]:
#number of records
len(data)

99

In [10]:
#understanding the json structure
print(data[1].keys())

for key in data[1].keys():
    print(key + '\n')
    print(data[1][key])
    print('\n')

dict_keys(['highestSeverity', 'notifications', 'duplicateWaybill', 'moreData', 'trackDetailsCount', 'trackDetails'])
highestSeverity

SUCCESS


notifications

[{'severity': 'SUCCESS', 'source': 'trck', 'code': '0', 'message': 'Request was successfully processed.', 'localizedMessage': 'Request was successfully processed.', 'messageParameters': []}]


duplicateWaybill

False


moreData

False


trackDetailsCount

0


trackDetails

[{'notification': {'severity': 'SUCCESS', 'source': 'trck', 'code': '0', 'message': 'Request was successfully processed.', 'localizedMessage': 'Request was successfully processed.', 'messageParameters': []}, 'trackingNumber': '390901883808', 'trackingNumberUniqueIdentifier': '2458915000~390901883808~FX', 'statusDetail': {'creationTime': {'$numberLong': '1583712000000'}, 'code': 'DL', 'description': 'Delivered', 'location': {'streetLines': [], 'city': 'Bangalore', 'stateOrProvinceCode': 'KA', 'countryCode': 'IN', 'countryName': 'India', 'residential': False}, 'a

In [16]:
#required data in 'trackDetails' field

td = data[1]['trackDetails']
print(td)

for tddata in td[0]:
    print(tddata)

[{'notification': {'severity': 'SUCCESS', 'source': 'trck', 'code': '0', 'message': 'Request was successfully processed.', 'localizedMessage': 'Request was successfully processed.', 'messageParameters': []}, 'trackingNumber': '390901883808', 'trackingNumberUniqueIdentifier': '2458915000~390901883808~FX', 'statusDetail': {'creationTime': {'$numberLong': '1583712000000'}, 'code': 'DL', 'description': 'Delivered', 'location': {'streetLines': [], 'city': 'Bangalore', 'stateOrProvinceCode': 'KA', 'countryCode': 'IN', 'countryName': 'India', 'residential': False}, 'ancillaryDetails': []}, 'informationNotes': [], 'customerExceptionRequests': [], 'carrierCode': 'FDXE', 'operatingCompanyOrCarrierDescription': 'FedEx Express', 'otherIdentifiers': [], 'service': {'type': 'FEDEX_EXPRESS_SAVER', 'description': 'FedEx Economy', 'shortDescription': 'XS'}, 'packageWeight': {'units': 'KG', 'value': 14}, 'shipmentWeight': {'units': 'KG', 'value': 14}, 'packaging': {'type': 'YOUR_PACKAGING', 'description

In [20]:
#payment present in specialhandlings field where type is COD, will use that for logic to find out if it is COD or prepaid
print(td[0]['specialHandlings'])

[{'type': 'DELIVER_WEEKDAY', 'description': 'Deliver Weekday', 'paymentType': 'OTHER'}]


In [37]:
#events have the delivery stages from picked up to delivered and details to compute picked-up to delivery
print(int(td[0]['events'][0]['timestamp']['$numberLong']))
print(td[0]['events'][-1]['timestamp']['$numberLong'])

1583763600000
1583476076000


In [55]:
from datetime import datetime
import pytz
#IST hence kolkata timezone
timezone = pytz.timezone('Asia/Kolkata')
#converting to int from str
dt = int(td[0]['events'][0]['timestamp']['$numberLong'])
print(dt)
#dividing by 1e3 to make it unix format
dt_obj = datetime.fromtimestamp(dt/1000,timezone)
#can use day to compute


1583763600000


In [72]:
#computing delivery time, pickup and drop locations

for event in td[0]["events"]:
    if event["eventType"] == "DL":
        delivery_date = int(event['timestamp']['$numberLong'])
        delivery_date_obj = datetime.fromtimestamp(delivery_date/1000,timezone)
        drop_pincode = event["address"]["postalCode"]
        drop_state = event["address"]["stateOrProvinceCode"]
        drop_city = event["address"]["city"]
        print(drop_pincode+', '+drop_city+', '+drop_state)
    if event["eventType"] == "PU":
        pickup_date = int(event['timestamp']['$numberLong'])
        pickup_date_obj = datetime.fromtimestamp(pickup_date/1000,timezone)
        pickup_pincode = event["address"]["postalCode"]
        pickup_state = event["address"]["stateOrProvinceCode"]
        pickup_city = event["address"]["city"]
        print(pickup_pincode+', '+pickup_city+', '+pickup_state)


delta = delivery_date_obj - pickup_date_obj
delta.days

560034, Bangalore, KA
560048, BANGALORE, KA


3

My assumptions for out for delivery and delivered
out for delivery using eventType = OD and eventType = DL for delivered

In [65]:
#deliveryAttempts
from collections import defaultdict
#using a dictionary to capture data with timestamp
delivery_attempts = defaultdict(int)
out_for_delivery_datetimes = []
for event in td[0]["events"]:
    timestamp = int(event["timestamp"]["$numberLong"]) // 1000
    #using utcfromtimestamp to avoid timezone stuff  
    event_date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')

    if event["eventType"] == "OD":
        #If Out For Delivery, mark it as a delivery attempt for the given date, and add it to the list
        delivery_attempts[event_date] += 1
        out_for_delivery_datetimes.append(datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))
    elif event["eventType"] == "DL":
        #If Delivered, check if Out For Delivery happened on the same date, and count as a delivery attempt
        out_for_delivery_date = (datetime.utcfromtimestamp(timestamp - 1)).strftime('%Y-%m-%d')
        if delivery_attempts[out_for_delivery_date] > 0:
            delivery_attempts[out_for_delivery_date] -= 1
        else:
            delivery_attempts[event_date] += 1

total_attempts = sum(delivery_attempts.values())

print("Number of delivery attempts:", total_attempts)
print("Out For Delivery Datetimes:", out_for_delivery_datetimes)


Number of delivery attempts: 3
Out For Delivery Datetimes: ['2020-03-09 05:20:00', '2020-03-09 04:55:00']
